In [1]:
device = 'cpu'
R = 1

In [ ]:
import torch
import torch.nn as nn
import numpy as np

from dataset import EmbDataset
from pathlib import Path

In [33]:
path = Path('C:/Users/Amine/Desktop/real features/saved_features/AdaptationProcessor/BiT-M-R50x1_in21k/BiT-M-R50x1_in21k_with_pca_256.npy')
dataset = EmbDataset(path, only_original=False)
data = dataset.load_all_to_device(device)

def euc_dist(a, b):
    return np.linalg.norm(a-b)

In [78]:
def density(embds, distance_func, radius):
    def calc_how_many_in_circle(embd, embds_list, radius):
        # s = 0
        # for i, e in enumerate(embds_list):
        #     if (distance_func(embd, e) <= radius) and not np.any(e != embd):
        #         s+=1

        # return s
        return sum([1 if distance_func(embd, e) <= radius and not torch.any(e != embd)
                      else 0 
                      for e in embds_list 
                   ])
        
    d = dict()
    d['left'], d['right'] = dict(), dict()

    for i, l_embd in enumerate(embds['left']):
        d['left'][data['label'][i]] = {'left_to_left'  : calc_how_many_in_circle(l_embd, 
                                                                                embds['left'], radius),
                                       'left_to_right' : calc_how_many_in_circle(l_embd, 
                                                                                embds['right'], radius),
        }

    for i, l_embd in enumerate(embds['right']):
        d['right'][data['label'][i]] = {'right_to_right'  : calc_how_many_in_circle(l_embd, 
                                                                                embds['right'], radius),
                                    #    
                                    # ,
        }

In [79]:
%%time
data2 = dict()
for k in data.keys():
    data2[k] = data[k][:100]

density(data2, euc_dist, R)

Wall time: 840 ms


In [80]:
%%time
torch.norm(data['left'][:100, None] - data['right'][:100], dim=2, p=2)
torch.norm(data['left'][:100, None] - data['left'][:100], dim=2, p=2) < R
torch.norm(data['right'][:100, None] - data['right'][:100], dim=2, p=2)

Wall time: 28.1 ms


tensor([[ True, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False,  True]])